<a href="https://colab.research.google.com/github/kashimmirza/K-Learning/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
#device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#input_size = 784 #28X28
hidden_size = 128
num_classes = 10
num_epochs = 2
batch_size =100
learning_rate =0.001
input_size = 28
sequence_length = 28
num_layers = 2

train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)
test_dataset = torchvision.datasets.MNIST(root='./data',
                                          train=False,
                                          transform= transforms.ToTensor())
#data loader
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                     batch_size=batch_size,
                                     shuffle=False)

class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
     super(RNN, self).__init__()
     self.num_layers = num_layers
     self.hidden_size = hidden_size
     self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
     self.fc = nn.Linear(hidden_size, num_classes)
     #x->batch_size, seq, input_size
  def forward(self, x):
      h0 = torch.zeros(self.num_layers,x.size(0), self.hidden_size).to(device)
      out, _ = self.rnn(x,h0)
        #out: batch_size, seq_lenth, hidden_size
        #out (N, 28, 128)
      out = out[:,-1,:]
        #out (N, 128)
      out = self.fc(out)
      return out
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        #origin shape: [100, 2, 28, 28]
        #resized: [100, 784]
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        #forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        #backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if(i+1)%100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [ {i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
      #Test the model
      #In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for images, labels in test_loader:
        image = images.reshape(-1,sequence_length, input_size ).to(device)
        labels = labels.to(device)
        outputs =model(images)
          #max return (value, index)
        _, predicted = torch.max(outputs.data,1)
        n_samples += labels.size(0)
        n_correct += (predicted ==labels).sum().item()
    acc = 100.0 * n_correct /n_samples
    print(f'Accuracy of the network on the 10000 test images: {acc} %')







Epoch [1/2], Step [ 100/600], Loss: 0.9852
Epoch [1/2], Step [ 200/600], Loss: 0.8006
Epoch [1/2], Step [ 300/600], Loss: 0.5970
Epoch [1/2], Step [ 400/600], Loss: 0.4180
Epoch [1/2], Step [ 500/600], Loss: 0.4406
Epoch [1/2], Step [ 600/600], Loss: 0.4878
Epoch [2/2], Step [ 100/600], Loss: 0.1924
Epoch [2/2], Step [ 200/600], Loss: 0.3262
Epoch [2/2], Step [ 300/600], Loss: 0.1629
Epoch [2/2], Step [ 400/600], Loss: 0.2539
Epoch [2/2], Step [ 500/600], Loss: 0.1457
Epoch [2/2], Step [ 600/600], Loss: 0.1920


RuntimeError: ignored